<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_algoritmos_001_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


 # **<font color="DeepPink">Modelos Supervisados: Regresión Lineal Simple</font>**

<p align="justify">
La regresión lineal es un método estadístico que trata de modelar la relación entre una <b>variable continua</b> y una o más variables independientes mediante el ajuste de una ecuación lineal. Se llama <b>regresión lineal simple</b> cuando solo hay una variable independiente y <b>regresión lineal múltiple</b> cuando hay más de una.
<br>
<br>
A la variable dependiente también se le conoce como variable explicada o variable respuesta, y a las variables independientes como regresoras, explicativas o <i>features</i>.
<br>
<br>
Suponga que llamamos $x$ a la variable explicativa o independiente e $y$ a la variable explicada o dependiente. Entonces, para cada valor de $x$, la variable explicada $y$ responde a la expresión de la forma:
<br>
<br>
$$ y = \beta_0+\beta_1x+\epsilon $$
<br>
<br>
Donde $\epsilon$ es una variable aleatoria con distribución normal $N(0,\sigma^2)$ y $\beta_0$ y $\beta_1$ son los parámetros del modelo.
<br>
<br>
Los supuestos que se analizan más adelante son 2:
<br>
<br>

* ***Normalidad*** *: cada variable aleatoria $\epsilon_i$, i=1,...,n tiene distribución normal.*
* ***Homoscedasticidad*** *(igualdad de varianzas): $var(\epsilon_1)=var(\epsilon_2)=...=var(\epsilon_n)$*

<p align="justify">
Por lo tanto, la regresión lineal es un método estadístico para predecir o inferir el valor que tomará la variable dependiente $y$ basado en los valores de una variable independiente $x$.
<br>
<br>
Se trata de encontrar la función lineal (con sus parámetros) que predice $y$ en función de la variable $x$.



<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Regresion-lineal-001.png?raw=true" width="500">
</p>


<p align="justify">
Para calcular la función lineal se puede utilizar el <b>método de mínimos cuadrados</b>. El cual busca la recta que minimiza la distancia total de los puntos (valor real) respecto de las predicciones.

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Regresion-lineal-002.png?raw=true" width="500">
</p>


 ## **<font color="DeepPink">Bibliotecas</font>**

In [1]:
import yfinance as yf #para obtener las series de precios históricos del fondo
import pandas_datareader.data as web #para obtener los factores del modelo de Fama & French
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

 ## **<font color="DeepPink">Conjunto de datos</font>**

👀 [Fidelity® Growth Company (FDGRX)](https://fundresearch.fidelity.com/mutual-funds/summary/316200104?type=sq-NavBar) es un fondo de inversiones que contiene más 500 compañías.

El siguiente proceso es explicado en el Colab [Datos remotos 001.01 de yahoo finanzas](https://campus.uda.edu.ar/mod/hvp/view.php?id=197654):

In [2]:
end = dt.date(2022, 12, 31)
start = dt.date(end.year - 20, end.month, end.day)
fund = ['FDGRX']
priceFund = yf.download(fund, start, end, interval = '1mo', rounding='true')['Adj Close']
retFund = priceFund.pct_change()[1:]
factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0]
factors = factors[2:]
retFund.index = factors.index
df = pd.merge(retFund, factors, on="Date")
df[['Mkt-RF','SMB','HML','RF']]=df[['Mkt-RF','SMB','HML','RF']]/100
df.rename(columns={"Adj Close":"Ri"}, inplace=True)
df['Ri-RF'] = df['Ri'] - df['RF']
df.drop(columns=["Ri", "RF"], inplace=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
df.head()

,Mkt-RF,SMB,HML,Ri-RF
Date,,,,
2003-02,-0.0188,-0.0045,-0.0138,-0.000900
2003-03,0.0109,0.0103,-0.0191,0.021989
2003-04,0.0822,0.0066,0.0117,0.083270
2003-05,0.0605,0.0474,0.0050,0.092364
2003-06,0.0142,0.0177,0.0015,0.008479


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 239 entries, 2003-02 to 2022-12
Freq: M
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  239 non-null    float64
 1   SMB     239 non-null    float64
 2   HML     239 non-null    float64
 3   Ri-RF   239 non-null    float64
dtypes: float64(4)
memory usage: 9.3 KB


<p align="justify">
El modelo de tres factores Fama-French es una extensión del modelo de valoración de activos de capital (CAPM por sus siglas en inglés). El modelo Fama-French tiene como objetivo explicar los rendimientos de las acciones a través de tres factores:</p>

- (1) prima por riesgo de mercado,
- (2) el rendimiento superior de las empresas de pequeña capitalización en relación con las empresas de gran capitalización bursatil, y
- (3) el rendimiento superior de las empresas de valor (valor libro - valor de mercado alto) en relación a las empresas de crecimiento (valor libro - valor de mercado bajo).

<p align="justify">
La lógica detrás del modelo es que las empresas de valor y de pequeña capitalización bursatil tienden a superar regularmente en cuanto a rendimiento esperado a las empresas de crecimiento y alta capitalización bursatil.<p>

La fórmula del modelo de tres factores es:

$$r_i = r_f + \beta_1(r_m - r_f) + \beta_2(SMB)+\beta_3(HML)+ϵ_i$$

donde:
* $r_i$  = tasa de rendimiento esperada
* $rf$ = tasa libre de riesgo
* $\beta$  = coeficiente del factor (sensibilidad)
* $(r_m – r_f)$ = prima de riesgo de mercado
* $SMB$ (Small Minus Big) = rendimiento excedente histórico de las empresas de pequeña capitalización bursatil sobre las empresas de gran capitalización. Efecto tamaño.
* $HML$ (High Minus Low) = exceso de rendimiento histórico de las acciones de valor sobre las acciones de crecimiento
* $\epsilon_i$  = error aleatorio

Más información sobre el modelo de Fama y French y la base de datos:
* [CFI: Fama-French Three-Factor Model](https://corporatefinanceinstitute.com/resources/valuation/fama-french-three-factor-model/)
* [Base de datos](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)
* [Pandas DataReader](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#fama-french)



In [5]:
df.rename(columns={'Mkt-RF':'PrimaRiesgoMercado',
                   'SMB':'EfectoTamaño',
                   'HML':'ValorCrecimiento',
                   'Ri-RF':'Rendimiento' }, inplace=True)

In [6]:
df.reset_index(drop=True, inplace=True)

In [7]:
df.head()

,PrimaRiesgoMercado,EfectoTamaño,ValorCrecimiento,Rendimiento
0,-0.0188,-0.0045,-0.0138,-0.000900
1,0.0109,0.0103,-0.0191,0.021989
2,0.0822,0.0066,0.0117,0.083270
3,0.0605,0.0474,0.0050,0.092364
4,0.0142,0.0177,0.0015,0.008479


 ## **<font color="DeepPink">División del conjunto de datos</font>**

<p align="justify">
👀 División del conjunto de entrenamiento y prueba.
</p>


In [8]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, random_state=123)

In [9]:
df_train.shape

(179, 4)

In [10]:
df_test.shape

(60, 4)

 # **<font color="DeepPink">Regresión Lineal Simple con `statmodels`</font>**

<p align="justify">
Suponga que un analista financiero quiere saber si existe una relación entre el <b>rendimiento del mercado</b> y el <b>rendimiento del activo financiero</b>. En caso de existir y de establecer un modelo, podría predecir el rendimiento del activo.

 ## **<font color="DeepPink">Representación Gráfica</font>**

In [11]:
px.scatter(df,
           x='PrimaRiesgoMercado',
           y='Rendimiento',
           title="Rendimiento del activo en función del rendimiento del mercado",
           template="gridon",
           trendline="ols",
           trendline_color_override="darkorange",
           labels={'PrimaRiesgoMercado':'Prima riesgo del Mercado'}
           )

<p align="justify">
En este gráfico se evidencia una significativa relación lineal entre la variable independiente y la variable dependiente, lo cual implica a priori una relación sólida y consistente entre ambas.

 ## **<font color="DeepPink">Correlación Lineal</font>**

In [12]:
from scipy.stats import pearsonr

<p align="justify">
La función <code>pearsonr</code> de la biblioteca <code>scipy.stats</code> en Python se utiliza para calcular el <b>coeficiente de correlación de Pearson</b> entre dos variables. El coeficiente de correlación de Pearson es una medida estadística que evalúa la fuerza y dirección de la relación lineal entre dos variables continuas. El resultado de pearsonr es un par de valores: el <b>coeficiente de correlación</b> y el <b>p-valor</b> asociado.

- El **coeficiente de correlación** de Pearson varía entre $-1$ y $1$. Un valor de $1$ indica una correlación positiva perfecta, lo que significa que las dos variables están perfectamente relacionadas en una relación lineal positiva. Un valor de $-1$ indica una correlación negativa perfecta, lo que implica una relación lineal negativa perfecta entre las variables. Un valor cercano a $0$ indica una correlación débil o inexistente entre las variables.

- El **p-valor** asociado proporciona una medida de la significancia estadística de la correlación calculada. Si el p-valor es menor que un umbral  predefinido (generalmente $0.05$), se considera que la correlación es estadísticamente significativa.

In [13]:
corr_test = pearsonr(x = df['PrimaRiesgoMercado'], y = df['Rendimiento'])
print("")
print("Coeficiente de correlación de Pearson: ", corr_test[0])
print("P-valor: ", corr_test[1])


Coeficiente de correlación de Pearson:  0.8627731033442474
P-valor:  3.785250543795665e-72


<p align="justify">
Al igual que el gráfico anterior, el test de correlación muestran una <b>relación lineal positiva de intensidad fuerte</b> (r = 0.86) y <b>estadísticamente significativa</b> (p-value = 3.785250543795665e-72). Tiene sentido intentar generar un modelo de regresión lineal con el objetivo de predecir el rendimiento del activo en función del rendimiento del mercado.

 ## **<font color="DeepPink">Ajuste del modelo</font>**

[Documentación](https://www.statsmodels.org/stable/index.html)

<p align="justify">
<code>Statsmodels</code> es un módulo de Python que proporciona clases y funciones para la estimación de modelos estadísticos, así como para realizar pruebas estadísticas y exploración de datos estadísticos. Hay disponible una lista extensa de estadísticas de resultados para cada estimador. Los resultados se comparan con los paquetes estadísticos existentes para garantizar que sean correctos. Este módulo tiene una sintaxis y características muy simulares a las del lenguaje <b>R</b>.
<br>
<br>
La implementación de regresión lineal de <code>Statsmodels</code>, es más completa que la de <code>Scikit-learn</code> ya que, además de ajustar el modelo, permite calcular los test estadísticos y análisis necesarios para verificar que se cumplen las condiciones sobre las que se basa este tipo de modelos.
<br>
<br>
<code>Statsmodels</code> tiene dos formas de entrenar el modelo:

- Indicando la fórmula del modelo y pasando los datos de entrenamiento como un dataframe que incluye la variable respuesta y los predictores. Esta forma es similar a la utilizada en **R**.

- Pasar dos matrices, una con los predictores y otra con la variable respuesta. Esta es igual a la empleada por `Scikit-learn`.

In [14]:
# bibliotecas utilizadas en este modelo:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

Creación del modelo utilizando el **modo fórmula** (similar a R).

In [15]:
model = smf.ols(formula="Rendimiento~PrimaRiesgoMercado", data=df_train)
model = model.fit()
print(model.summary());

                            OLS Regression Results                            
Dep. Variable:            Rendimiento   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     471.2
Date:                Tue, 27 Jun 2023   Prob (F-statistic):           8.97e-52
Time:                        02:38:51   Log-Likelihood:                 368.88
No. Observations:                 179   AIC:                            -733.8
Df Residuals:                     177   BIC:                            -727.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.0012      0

<p align="justify">
El modelo tiene un $𝑅^2$ alto (0.727). Es decir, <code>PrimaRiesgoMercado</code> es capaz de explicar el $72,7$% de la variabilidad observada en el <code>Rendimiento</code>.
<br>
<br>
F estadísitco grande (F-statistic =	471.2) y p-valor asociado extremadamente pequeño (Prob (F-statistic) = 8.97e-52), el modelo en su conjunto es estadísticamente significativo.
<br>
<br>
La constante (<code>Intercept</code>) no es estadísticamente significativa. La variable si es es estadísticamente significativa. Es decir, <code>PrimaRiesgoMercado</code> explican significativamente el <code>Rendimiento</code> de un activo financiero. Dicho de otro modo, el rendimiento del mercado explica el rendimiento de un activo.

 ## **<font color="DeepPink">Interpretación</font>**

In [16]:
model.params #parámetros (coeficientes) del modelo generado.

Intercept             0.001180
PrimaRiesgoMercado    1.172052
dtype: float64

$$Rendimiento = 1.172052\ PrimaRiesgoMercado$$

<p align="justify">
El coeficiente $\beta_1$ de $1.172052$ indica que cuando el rendimiento de mercado aumenta en un $1$% el rendimiento del activo lo hace en un $1.172052$%. Este coeficiente en la literatura financiera se conoce como "Coeficiente Beta".


 ## **<font color="DeepPink">Predicción</font>**

In [17]:
df_test.reset_index(drop=True, inplace=True)

In [18]:
X_test = df_test[['PrimaRiesgoMercado']]
y_test = df_test['Rendimiento']

<p align="justify">
Una vez entrenado el modelo, se pueden obtener predicciones para nuevos datos. Los modelos de <code>statsmodels</code> permiten calcular las predicciones de dos formas:

- `predict()`: devuelve únicamente el valor de las predicciones.

- `get_prediction().summary_frame()`: devuelve, además de las predicciones, los intervalos de confianza asociados.

In [19]:
prediction = model.predict(X_test)

In [20]:
tabla = pd.DataFrame({"Prediccion":prediction,
                      "Real":y_test,
                      "Residuos": (y_test-prediction),
                      "Rendimiento_medio": y_test.mean()})
tabla.head()

,Prediccion,Real,Residuos,Rendimiento_medio
0,0.050172,0.024768,-0.025404,0.017176
1,0.022043,0.034425,0.012382,0.017176
2,0.055095,0.075948,0.020853,0.017176
3,-0.155757,-0.103817,0.051940,0.017176
4,0.049000,0.061389,0.012389,0.017176


In [21]:
fig = go.Figure([go.Scatter(x = X_test.PrimaRiesgoMercado,
                            y = y_test,
                            mode = "markers",
                            showlegend = False,
                            name = "Real"),
                 go.Scatter(x = X_test.PrimaRiesgoMercado,
                            y = prediction,
                            name = "Regresión Lineal")
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Regresion lineal simple",
                  yaxis_title = "Rendimiento del activo financiero",
                  xaxis_title = "Rendimiento del mercado")

fig.show()

<p align="justify">
Además de la línea de mínimos cuadrados, es recomendable incluir los límites superior e inferior del intervalo de confianza. Esto permite identificar la región en la que, según el modelo generado y para un determinado nivel de confianza, se encuentra el valor promedio de la variable respuesta.

In [22]:
prediction2 = model.get_prediction(exog = X_test).summary_frame(alpha=0.05)
prediction2['x'] = X_test.PrimaRiesgoMercado
prediction2['y'] = y_test
prediction2 = prediction2.sort_values('x')
prediction2.head()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,x,y
3,-0.155757,0.007940,-0.171427,-0.140088,-0.218889,-0.092626,-0.1339,-0.103817
55,-0.117197,0.006263,-0.129556,-0.104838,-0.179589,-0.054804,-0.1010,-0.070933
20,-0.108406,0.005888,-0.120027,-0.096786,-0.170657,-0.046156,-0.0935,-0.101289
56,-0.093990,0.005284,-0.104418,-0.083562,-0.156029,-0.031951,-0.0812,-0.036145
27,-0.088833,0.005072,-0.098842,-0.078824,-0.150803,-0.026863,-0.0768,-0.123253


In [23]:
fig = go.Figure([go.Scatter(x = prediction2.x,
                            y = prediction2["mean"],
                            name = "Predicción media (OLS)"),
                 go.Scatter(x = prediction2.x,
                            y = prediction2.mean_ci_upper,
                            name = "Limite superior"),
                 go.Scatter(x = prediction2.x,
                            y = prediction2.mean_ci_lower,
                            name = "Limite inferior"),
                 go.Scatter(x = prediction2.x,
                            y = prediction2.y,
                            mode = "markers",
                            showlegend = False,
                            name = "Real")
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Regresion lineal simple",
                  yaxis_title = "Rendimiento del activo financiero",
                  xaxis_title = "Rendimiento del mercado")

fig.show()

 ## **<font color="DeepPink">Evaluación del modelo</font>**

**<font color="DeepPink">Standardised mean squared error (SMSE):**

$$SMSE = \sqrt{MSE}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y})^2}$$

In [24]:
SMSE = np.sqrt(sum((tabla.Real-tabla.Prediccion)**2)/(len(tabla))).round(2)
SMSE

0.02

**<font color="DeepPink">Mean Absolute Error(MAE):**

$$MAE=\frac{1}{n}\sum_{i=1}^{n}|y_i-\hat{y}|$$

In [25]:
MAE = round(sum(abs(tabla.Real-tabla.Prediccion))/(len(tabla)),2)
MAE

0.02

**<font color="DeepPink">$R_2$:**


$$R_2 = 1-\frac{\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y})^2}{\frac{1}{n}\sum_{i=1}^{n}(y_i-\bar{y})^2}$$

Donde:
* $\hat{y}$ es la predicción de la variable respuesta,
* $\bar{y}$ es el promedio de la variable respuesta.

In [26]:
R2= round(1-((sum((tabla.Real-tabla.Prediccion)**2)/(len(tabla)))/(sum((tabla.Real-tabla.Rendimiento_medio)**2)/(len(tabla)))),2)
R2

0.8

In [27]:
fig = go.Figure([go.Scatter(x = X_test.PrimaRiesgoMercado,
                            y = y_test,
                            mode = "markers",
                            showlegend = False,
                            name = "Real"),
                 go.Scatter(x = X_test.PrimaRiesgoMercado,
                            y = tabla.Prediccion,
                            name = "Predicción media (OLS)"),
                 go.Scatter(x = X_test.PrimaRiesgoMercado,
                            y = tabla.Rendimiento_medio,
                            name = "Rendimiento medio")
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Regresion lineal simple",
                  yaxis_title = "Rendimiento del activo financiero",
                  xaxis_title = "Rendimiento del mercado")

fig.show()

Podemos calcular las métricas anteriores (calculadas manualmente) usando el módulo `metrics` de `scikit-learn`:

In [28]:
from sklearn import metrics
SMSE = round(metrics.mean_squared_error(y_test, prediction,squared=False),2) #Error cuadrático medio
MAE = round(metrics.mean_absolute_error(y_test, prediction),2) #Error absoluto medio
R2 = round(metrics.r2_score(y_test, prediction),4)
print("")
print("SMSE: {}".format(SMSE))
print("MAE: {}".format(MAE))
print("R2: {}".format(R2))


SMSE: 0.02
MAE: 0.02
R2: 0.8017


 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:<br><br>
✅
Estudiamos la correlación lineal entre variables. <br>
✅ Vimos como entrenar un modelo de regresión lineal simple usando la biblioteca <code>Statsmodels</code>. <br>✅ Realizamos la predicción y evaluación, usando diferentes métricas, con un conjunto de prueba. <br>

<p align="justify">



<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
